In [155]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 100)

In [156]:
def asset_growth(ticker):
    """Asset growth is defined as the YoY % change in quarterly total assets.
    Higher is worse, so multiply by -1"""
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df = df[::-1]
    df['Asset_growth'] = df.groupby('Quarter')['Total Assets'].pct_change()*-1
    return df[['Calendar_Year', 'Quarter','Total Assets','Asset_growth']]

In [157]:
asset_growth('AAPL').tail()

,Calendar_Year,Quarter,Total Assets,Asset_growth
4,2021,Q3,"329,840.00",-0.04
3,2021,Q4,"351,002.00",-0.08
2,2022,Q1,"381,191.00",-0.08
1,2022,Q2,"350,662.00",-0.04
0,2022,Q3,"336,309.00",-0.02


In [158]:
def asset_turnover_change(ticker):
    """YoY % change in the ratio of Sales to Total Assets. Higher is better"""
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df = df[::-1]
    
    def divide_two_cols(df):
        df['Asset_Turnover'] = df['Revenue'] / (df['Total Assets'])
        return df
        
    df = divide_two_cols(df)
    df['Asset_Turnover_chg'] = df.groupby('Quarter')['Asset_Turnover'].pct_change()
    
    df['Asset_growth'] = df.groupby('Quarter')['Total Assets'].pct_change()*-1

    return df

In [159]:
df_AAPL = asset_turnover_change('AAPL')
df_AAPL.tail()

,Unnamed: 0,Calendar_Year,Quarter,Mkt Cap,Debt to Equity,Debt to Assets,Revenue per Share,NI per Share,outstandingShares,Revenue,Gross Profit,R&D Expenses,Op Expenses,Op Income,Net Income,Cash,Receivables,Inventory,Cur Assets,Tax Assets,LT Assets,Int Assets,Total Assets,Payables,Deferred Revenue Current,Deferred Revenue LongTerm,Cur Liab,LT Debt,LT Liab,Total Liab,SH Equity,cash_flow Operations,cash_flow Investing,cash_flow Financing,CAPEX,Fcash_flow,Dividends Paid,Gross Profit Margin,Op Margin,Int Coverage,Net Profit Margin,Dividend Yield,Current Ratio,Operating Cycle,Days of AP Outstanding,Cash Conversion Cycle,ROA,ROE,ROCE,PE,PS,PB,Price To FCF,PEG,EPS,Ticker,Asset_Turnover,Asset_Turnover_chg,Asset_growth
4,2021-06-26,2021,Q3,"2,030,688.81",1.57,0.79,5.35,1.41,"16,781.74","81,434.00","35,255.00","5,717.00","11,129.00","24,126.00","21,744.00","34,050.00","33,908.00","5,178.00","114,423.00",0.00,"215,417.00",0.00,"329,840.00","40,409.00","7,681.00",0.00,"107,754.00","105,752.00","157,806.00","265,560.00","64,280.00","23,981.00","-10,368.00","-11,326.00","-2,269.00","21,712.00","-3,447.00",0.43,0.31,41.05,0.26,0.00,1.14,42.31,70.12,-27.81,0.07,0.34,0.12,21.48,22.67,29.35,93.53,-1.26,1.31,AAPL,0.25,0.31,-0.04
3,2021-09-25,2021,Q4,"2,213,535.58",1.65,0.81,4.90,1.31,"16,635.10","83,360.00","35,174.00","5,772.00","11,388.00","23,786.00","20,551.00","34,940.00","51,506.00","6,580.00","134,836.00",0.00,"216,166.00",0.00,"351,002.00","54,763.00","7,612.00",0.00,"125,481.00","109,106.00","162,431.00","287,912.00","63,090.00","21,094.00","3,572.00","-29,396.00","-2,093.00","19,001.00","-3,767.00",0.43,0.30,36.28,0.27,0.00,1.06,47.57,78.75,-31.19,0.07,0.34,0.11,25.45,27.18,34.44,116.50,-3.59,1.25,AAPL,0.24,0.19,-0.08
2,2021-12-25,2022,Q1,"2,422,287.79",1.73,0.82,5.06,1.25,"16,519.29","123,945.00","54,243.00","6,306.00","12,755.00","41,488.00","34,630.00","37,119.00","65,253.00","5,876.00","153,154.00",0.00,"228,037.00",0.00,"381,191.00","74,362.00","7,876.00",0.00,"147,574.00","106,629.00","161,685.00","309,259.00","71,932.00","20,200.00",835.00,"-20,382.00","-3,223.00","16,977.00","-3,640.00",0.42,0.29,35.40,0.25,0.00,1.07,67.90,102.28,-34.39,0.06,0.33,0.11,29.47,29.06,38.39,142.68,-6.43,2.11,AAPL,0.33,0.03,-0.08
1,2022-03-26,2022,Q2,"2,955,919.62",1.48,0.81,7.56,2.11,"16,519.29","97,278.00","42,559.00","6,387.00","12,580.00","29,979.00","25,010.00","28,098.00","45,400.00","5,460.00","118,180.00",0.00,"232,482.00",0.00,"350,662.00","52,682.00","7,920.00",0.00,"127,508.00","103,323.00","155,755.00","283,263.00","67,399.00","46,966.00","-16,106.00","-28,159.00","-2,803.00","44,163.00","-3,732.00",0.44,0.33,59.78,0.28,0.00,1.04,54.97,96.02,-41.05,0.09,0.48,0.18,21.34,23.85,41.09,66.93,0.31,1.53,AAPL,0.28,0.04,-0.04
0,2022-06-25,2022,Q3,"2,863,962.04",1.53,0.81,5.93,1.53,"16,262.20","82,959.00","35,885.00","6,797.00","12,809.00","23,076.00","19,442.00","27,502.00","42,242.00","5,433.00","112,292.00",0.00,"224,017.00",0.00,"336,309.00","48,343.00","7,728.00",0.00,"129,873.00","94,700.00","148,329.00","278,202.00","58,107.00","28,166.00","-9,265.00","-28,351.00","-2,514.00","25,652.00","-3,595.00",0.44,0.31,43.38,0.26,0.00,0.93,50.98,86.65,-35.67,0.07,0.37,0.13,28.63,29.44,42.49,111.65,-1.04,1.20,AAPL,0.25,-0.00,-0.02


In [160]:
def cap_spend_growth(ticker):
    """YoY % change of: (Latest 4 quarters of cap ex + R&D expense)/(latest 4 quarters of sales)*-1"""
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df = df[::-1]
    df['TTM_expenses'] = (df['R&D Expenses'].rolling(4).sum()+df['CAPEX'].rolling(4).sum()*-1)
    df['TTM_sales'] = df['Revenue'].rolling(4).sum()
    df['cap_spend_%_of_rev'] = df['TTM_expenses']/df['TTM_sales']
    df['cap_spend_growth'] = df.groupby('Quarter')['cap_spend_%_of_rev'].pct_change()
    return df
    

In [161]:
cap_spend_growth('AAPL').tail()

,Unnamed: 0,Calendar_Year,Quarter,Mkt Cap,Debt to Equity,Debt to Assets,Revenue per Share,NI per Share,outstandingShares,Revenue,Gross Profit,R&D Expenses,Op Expenses,Op Income,Net Income,Cash,Receivables,Inventory,Cur Assets,Tax Assets,LT Assets,Int Assets,Total Assets,Payables,Deferred Revenue Current,Deferred Revenue LongTerm,Cur Liab,LT Debt,LT Liab,Total Liab,SH Equity,cash_flow Operations,cash_flow Investing,cash_flow Financing,CAPEX,Fcash_flow,Dividends Paid,Gross Profit Margin,Op Margin,Int Coverage,Net Profit Margin,Dividend Yield,Current Ratio,Operating Cycle,Days of AP Outstanding,Cash Conversion Cycle,ROA,ROE,ROCE,PE,PS,PB,Price To FCF,PEG,EPS,Ticker,TTM_expenses,TTM_sales,cap_spend_%_of_rev,cap_spend_growth
4,2021-06-26,2021,Q3,"2,030,688.81",1.57,0.79,5.35,1.41,"16,781.74","81,434.00","35,255.00","5,717.00","11,129.00","24,126.00","21,744.00","34,050.00","33,908.00","5,178.00","114,423.00",0.00,"215,417.00",0.00,"329,840.00","40,409.00","7,681.00",0.00,"107,754.00","105,752.00","157,806.00","265,560.00","64,280.00","23,981.00","-10,368.00","-11,326.00","-2,269.00","21,712.00","-3,447.00",0.43,0.31,41.05,0.26,0.00,1.14,42.31,70.12,-27.81,0.07,0.34,0.12,21.48,22.67,29.35,93.53,-1.26,1.31,AAPL,"30,238.00","347,155.00",0.09,-0.10
3,2021-09-25,2021,Q4,"2,213,535.58",1.65,0.81,4.90,1.31,"16,635.10","83,360.00","35,174.00","5,772.00","11,388.00","23,786.00","20,551.00","34,940.00","51,506.00","6,580.00","134,836.00",0.00,"216,166.00",0.00,"351,002.00","54,763.00","7,612.00",0.00,"125,481.00","109,106.00","162,431.00","287,912.00","63,090.00","21,094.00","3,572.00","-29,396.00","-2,093.00","19,001.00","-3,767.00",0.43,0.30,36.28,0.27,0.00,1.06,47.57,78.75,-31.19,0.07,0.34,0.11,25.45,27.18,34.44,116.50,-3.59,1.25,AAPL,"31,560.00","365,817.00",0.09,-0.12
2,2021-12-25,2022,Q1,"2,422,287.79",1.73,0.82,5.06,1.25,"16,519.29","123,945.00","54,243.00","6,306.00","12,755.00","41,488.00","34,630.00","37,119.00","65,253.00","5,876.00","153,154.00",0.00,"228,037.00",0.00,"381,191.00","74,362.00","7,876.00",0.00,"147,574.00","106,629.00","161,685.00","309,259.00","71,932.00","20,200.00",835.00,"-20,382.00","-3,223.00","16,977.00","-3,640.00",0.42,0.29,35.40,0.25,0.00,1.07,67.90,102.28,-34.39,0.06,0.33,0.11,29.47,29.06,38.39,142.68,-6.43,2.11,AAPL,"34,142.00","378,323.00",0.09,-0.01
1,2022-03-26,2022,Q2,"2,955,919.62",1.48,0.81,7.56,2.11,"16,519.29","97,278.00","42,559.00","6,387.00","12,580.00","29,979.00","25,010.00","28,098.00","45,400.00","5,460.00","118,180.00",0.00,"232,482.00",0.00,"350,662.00","52,682.00","7,920.00",0.00,"127,508.00","103,323.00","155,755.00","283,263.00","67,399.00","46,966.00","-16,106.00","-28,159.00","-2,803.00","44,163.00","-3,732.00",0.44,0.33,59.78,0.28,0.00,1.04,54.97,96.02,-41.05,0.09,0.48,0.18,21.34,23.85,41.09,66.93,0.31,1.53,AAPL,"34,570.00","386,017.00",0.09,0.01
0,2022-06-25,2022,Q3,"2,863,962.04",1.53,0.81,5.93,1.53,"16,262.20","82,959.00","35,885.00","6,797.00","12,809.00","23,076.00","19,442.00","27,502.00","42,242.00","5,433.00","112,292.00",0.00,"224,017.00",0.00,"336,309.00","48,343.00","7,728.00",0.00,"129,873.00","94,700.00","148,329.00","278,202.00","58,107.00","28,166.00","-9,265.00","-28,351.00","-2,514.00","25,652.00","-3,595.00",0.44,0.31,43.38,0.26,0.00,0.93,50.98,86.65,-35.67,0.07,0.37,0.13,28.63,29.44,42.49,111.65,-1.04,1.20,AAPL,"35,895.00","387,542.00",0.09,0.06


In [162]:
def cap_spend_level(ticker):
    """(latest 4 quarters of cap ex + R&D expense)/latest 4 qtrs of sales*-1"""
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df = df[::-1]
    df['TTM_expenses'] = (df['R&D Expenses'].rolling(4).sum() +df['CAPEX'].rolling(4).sum()*-1)
    df['TTM_sales'] = df['Revenue'].rolling(4).sum()
    df['cap_spend_%_of_rev'] = df['TTM_expenses']/df['TTM_sales']
    return df[::-1]

In [163]:
cap_spend_level('AAPL').head(10)

,Unnamed: 0,Calendar_Year,Quarter,Mkt Cap,Debt to Equity,Debt to Assets,Revenue per Share,NI per Share,outstandingShares,Revenue,Gross Profit,R&D Expenses,Op Expenses,Op Income,Net Income,Cash,Receivables,Inventory,Cur Assets,Tax Assets,LT Assets,Int Assets,Total Assets,Payables,Deferred Revenue Current,Deferred Revenue LongTerm,Cur Liab,LT Debt,LT Liab,Total Liab,SH Equity,cash_flow Operations,cash_flow Investing,cash_flow Financing,CAPEX,Fcash_flow,Dividends Paid,Gross Profit Margin,Op Margin,Int Coverage,Net Profit Margin,Dividend Yield,Current Ratio,Operating Cycle,Days of AP Outstanding,Cash Conversion Cycle,ROA,ROE,ROCE,PE,PS,PB,Price To FCF,PEG,EPS,Ticker,TTM_expenses,TTM_sales,cap_spend_%_of_rev
0,2022-06-25,2022,Q3,"2,863,962.04",1.53,0.81,5.93,1.53,"16,262.20","82,959.00","35,885.00","6,797.00","12,809.00","23,076.00","19,442.00","27,502.00","42,242.00","5,433.00","112,292.00",0.00,"224,017.00",0.00,"336,309.00","48,343.00","7,728.00",0.00,"129,873.00","94,700.00","148,329.00","278,202.00","58,107.00","28,166.00","-9,265.00","-28,351.00","-2,514.00","25,652.00","-3,595.00",0.44,0.31,43.38,0.26,0.00,0.93,50.98,86.65,-35.67,0.07,0.37,0.13,28.63,29.44,42.49,111.65,-1.04,1.20,AAPL,"35,895.00","387,542.00",0.09
1,2022-03-26,2022,Q2,"2,955,919.62",1.48,0.81,7.56,2.11,"16,519.29","97,278.00","42,559.00","6,387.00","12,580.00","29,979.00","25,010.00","28,098.00","45,400.00","5,460.00","118,180.00",0.00,"232,482.00",0.00,"350,662.00","52,682.00","7,920.00",0.00,"127,508.00","103,323.00","155,755.00","283,263.00","67,399.00","46,966.00","-16,106.00","-28,159.00","-2,803.00","44,163.00","-3,732.00",0.44,0.33,59.78,0.28,0.00,1.04,54.97,96.02,-41.05,0.09,0.48,0.18,21.34,23.85,41.09,66.93,0.31,1.53,AAPL,"34,570.00","386,017.00",0.09
2,2021-12-25,2022,Q1,"2,422,287.79",1.73,0.82,5.06,1.25,"16,519.29","123,945.00","54,243.00","6,306.00","12,755.00","41,488.00","34,630.00","37,119.00","65,253.00","5,876.00","153,154.00",0.00,"228,037.00",0.00,"381,191.00","74,362.00","7,876.00",0.00,"147,574.00","106,629.00","161,685.00","309,259.00","71,932.00","20,200.00",835.00,"-20,382.00","-3,223.00","16,977.00","-3,640.00",0.42,0.29,35.40,0.25,0.00,1.07,67.90,102.28,-34.39,0.06,0.33,0.11,29.47,29.06,38.39,142.68,-6.43,2.11,AAPL,"34,142.00","378,323.00",0.09
3,2021-09-25,2021,Q4,"2,213,535.58",1.65,0.81,4.90,1.31,"16,635.10","83,360.00","35,174.00","5,772.00","11,388.00","23,786.00","20,551.00","34,940.00","51,506.00","6,580.00","134,836.00",0.00,"216,166.00",0.00,"351,002.00","54,763.00","7,612.00",0.00,"125,481.00","109,106.00","162,431.00","287,912.00","63,090.00","21,094.00","3,572.00","-29,396.00","-2,093.00","19,001.00","-3,767.00",0.43,0.30,36.28,0.27,0.00,1.06,47.57,78.75,-31.19,0.07,0.34,0.11,25.45,27.18,34.44,116.50,-3.59,1.25,AAPL,"31,560.00","365,817.00",0.09
4,2021-06-26,2021,Q3,"2,030,688.81",1.57,0.79,5.35,1.41,"16,781.74","81,434.00","35,255.00","5,717.00","11,129.00","24,126.00","21,744.00","34,050.00","33,908.00","5,178.00","114,423.00",0.00,"215,417.00",0.00,"329,840.00","40,409.00","7,681.00",0.00,"107,754.00","105,752.00","157,806.00","265,560.00","64,280.00","23,981.00","-10,368.00","-11,326.00","-2,269.00","21,712.00","-3,447.00",0.43,0.31,41.05,0.26,0.00,1.14,42.31,70.12,-27.81,0.07,0.34,0.12,21.48,22.67,29.35,93.53,-1.26,1.31,AAPL,"30,238.00","347,155.00",0.09
5,2021-03-27,2021,Q2,"2,314,861.46",1.50,0.81,6.58,1.70,"16,929.16","89,584.00","38,079.00","5,262.00","10,576.00","27,503.00","23,630.00","38,466.00","33,036.00","5,219.00","121,465.00",0.00,"215,693.00",0.00,"337,158.00","40,127.00","7,595.00",0.00,"106,385.00","108,642.00","161,595.00","267,980.00","69,178.00","38,763.00","-8,584.00","-32,249.00","-3,500.00","35,263.00","-3,613.00",0.40,0.30,52.56,0.26,0.00,1.16,54.01,85.62,-31.61,0.08,0.43,0.15,20.13,20.77,34.96,65.65,0.16,1.41,AAPL,"28,863.00","325,406.00",0.09
6,2020-12-26,2021,Q1,"1,915,229.78",1.51,0.80,3.79,0.74,"17,113.69","111,439.00","44,328.00","5,163.00","10,794.00","33,534.00","28,755.00","36,010.00"

In [164]:
def working_capital_accruals(ticker):
    """Trailing 4 quarters: 
    chg in receivables + chg in inventories + chg in accts payable + chg in tax accruals + change in misc. assets and liabilites
    Scaled by avg Total Assets
    """
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df = df[::-1]
    df['chgReceivables'] = df.groupby(['Quarter'])['Receivables'].diff().fillna(0)      
    df['chgInventory'] = df.groupby(['Quarter'])['Inventory'].diff().fillna(0)                      
    df['chgPayables'] = df.groupby(['Quarter'])['Payables'].diff().fillna(0)  
    df['chgTaxAcc'] = df.groupby(['Quarter'])['Tax Assets'].diff().fillna(0)
    #df['chgMisc'] = 
    df['Accruals'] = df['chgReceivables'] + df['chgInventory'] + df['chgPayables'] + df['chgTaxAcc']
    df['avgTotalAssets'] = df['Total Assets'].rolling(4).mean().fillna(0)
    df['avgAssetAccruals'] = (df['Accruals'] / df['avgTotalAssets']).fillna(0)
    return df
    

In [165]:
working_capital_accruals('AAPL').tail(3)

,Unnamed: 0,Calendar_Year,Quarter,Mkt Cap,Debt to Equity,Debt to Assets,Revenue per Share,NI per Share,outstandingShares,Revenue,Gross Profit,R&D Expenses,Op Expenses,Op Income,Net Income,Cash,Receivables,Inventory,Cur Assets,Tax Assets,LT Assets,Int Assets,Total Assets,Payables,Deferred Revenue Current,Deferred Revenue LongTerm,Cur Liab,LT Debt,LT Liab,Total Liab,SH Equity,cash_flow Operations,cash_flow Investing,cash_flow Financing,CAPEX,Fcash_flow,Dividends Paid,Gross Profit Margin,Op Margin,Int Coverage,Net Profit Margin,Dividend Yield,Current Ratio,Operating Cycle,Days of AP Outstanding,Cash Conversion Cycle,ROA,ROE,ROCE,PE,PS,PB,Price To FCF,PEG,EPS,Ticker,chgReceivables,chgInventory,chgPayables,chgTaxAcc,Accruals,avgTotalAssets,avgAssetAccruals
2,2021-12-25,2022,Q1,"2,422,287.79",1.73,0.82,5.06,1.25,"16,519.29","123,945.00","54,243.00","6,306.00","12,755.00","41,488.00","34,630.00","37,119.00","65,253.00","5,876.00","153,154.00",0.00,"228,037.00",0.00,"381,191.00","74,362.00","7,876.00",0.00,"147,574.00","106,629.00","161,685.00","309,259.00","71,932.00","20,200.00",835.00,"-20,382.00","-3,223.00","16,977.00","-3,640.00",0.42,0.29,35.40,0.25,0.00,1.07,67.90,102.28,-34.39,0.06,0.33,0.11,29.47,29.06,38.39,142.68,-6.43,2.11,AAPL,"6,633.00",903.00,"10,516.00",0.00,"18,052.00","349,797.75",0.05
1,2022-03-26,2022,Q2,"2,955,919.62",1.48,0.81,7.56,2.11,"16,519.29","97,278.00","42,559.00","6,387.00","12,580.00","29,979.00","25,010.00","28,098.00","45,400.00","5,460.00","118,180.00",0.00,"232,482.00",0.00,"350,662.00","52,682.00","7,920.00",0.00,"127,508.00","103,323.00","155,755.00","283,263.00","67,399.00","46,966.00","-16,106.00","-28,159.00","-2,803.00","44,163.00","-3,732.00",0.44,0.33,59.78,0.28,0.00,1.04,54.97,96.02,-41.05,0.09,0.48,0.18,21.34,23.85,41.09,66.93,0.31,1.53,AAPL,"12,364.00",241.00,"12,555.00",0.00,"25,160.00","353,173.75",0.07
0,2022-06-25,2022,Q3,"2,863,962.04",1.53,0.81,5.93,1.53,"16,262.20","82,959.00","35,885.00","6,797.00","12,809.00","23,076.00","19,442.00","27,502.00","42,242.00","5,433.00","112,292.00",0.00,"224,017.00",0.00,"336,309.00","48,343.00","7,728.00",0.00,"129,873.00","94,700.00","148,329.00","278,202.00","58,107.00","28,166.00","-9,265.00","-28,351.00","-2,514.00","25,652.00","-3,595.00",0.44,0.31,43.38,0.26,0.00,0.93,50.98,86.65,-35.67,0.07,0.37,0.13,28.63,29.44,42.49,111.65,-1.04,1.20,AAPL,"8,334.00",255.00,"7,934.00",0.00,"16,523.00","354,791.00",0.05


## Momentum

In [166]:
def RelativeStrengthIndex(ticker, time_window):
    """RSI compares the magnitude of recent price changes across stocks
    to identify stocks as overbought or oversold.
    A high RSI (usually above 70) indicate overbought and a low RSI 
    (typically below 30) indicates oversold.
    If first computes the avg price change for a given # (often 14)
    of peior trading days with rising prices and falling prices"""
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/prices/{ticker}.csv')
    df = df[::-1]
    diff = df['change'].dropna()
    
    # preserves dimensions of diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change  is equal to the positive diff, otherwise 0
    up_chg[diff > 0] = diff[diff >0]
    
    # down chg is equal to negative diff, oterhwise 0
    down_chg[diff < 0] = diff[diff < 0]
    
    up_chg_avg = up_chg.ewm(com=time_window-1, min_periods = time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1, min_periods = time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi
    
    
    

In [167]:
RelativeStrengthIndex('AAPL',14).to_frame().tail()

,change
4,67.73
3,65.13
2,63.50
1,68.86
0,69.28


In [168]:
aapl = pd.read_csv('/Users/brendan/Desktop/Python/BoostedAI/prices/AAPL.csv')
aapl['RSI'] = RelativeStrengthIndex('AAPL',14)
aapl.head()


,Unnamed: 0,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,Ticker,RSI
0,0,2022-07-28,156.98,157.64,154.41,157.35,157.35,"66,928,784.00","66,928,784.00",0.37,0.24,156.47,"July 28, 22",0.00,AAPL,69.28
1,1,2022-07-27,152.58,157.33,152.16,156.79,156.79,"78,464,782.00","78,464,782.00",4.21,2.76,155.43,"July 27, 22",0.03,AAPL,68.86
2,2,2022-07-26,152.26,153.09,150.80,151.60,151.60,"54,828,007.00","54,828,007.00",-0.66,-0.43,151.83,"July 26, 22",-0.00,AAPL,63.50
3,3,2022-07-25,154.01,155.04,152.28,152.95,152.95,"53,518,694.00","53,518,694.00",-1.06,-0.69,153.42,"July 25, 22",-0.01,AAPL,65.13
4,4,2022-07-22,155.39,156.28,153.41,154.09,154.09,"66,626,929.00","66,626,929.00",-1.30,-0.84,154.59,"July 22, 22",-0.01,AAPL,67.73


In [169]:
def price_momentum(ticker):
    """This factor computes the total return for a given number of prior days.
        It is common to use the last 12 months except for the most recent month
    """
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/prices/{ticker}.csv')
    df = df[::-1]
    df['Momentum'] = df['changePercent'].shift(1)/df['changePercent'].shift(12)-1
    return df
    

In [170]:
def price_momentum_vol_adj(ticker):
    """The indicator normalizes the total return over the previous 12 months by dividing it by the standard deviation of these returns."""
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/prices/{ticker}.csv')
    df = df[::-1]
    df['MoVolAdj'] = (df['adjClose'].shift(0)/df['adjClose'].shift(252)-1)/df['changePercent'].rolling(252).std()
    return df

In [171]:
aapl = price_momentum_vol_adj('AAPL')
len(aapl.sort_values(by=['MoVolAdj']).dropna())

1007

## Value

In [172]:
def adj_book_to_price(ticker):
    """"The Residual (Actual - Estimate) after regressing Book-to-price (depend var) over ROE (independent var).
    Note: Total Equity is adjusted for Intangibles and Goodwill
    """
    df_price = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/prices/{ticker}.csv')
    #df = df[::-1]
    df_data = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df_merged = pd.concat([df_price,df_data], axis=1)
    return df_merged

In [173]:
adj_book_to_price('AVGO').head()

,Unnamed: 0,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,Ticker,Unnamed: 0,Calendar_Year,Quarter,Mkt Cap,Debt to Equity,Debt to Assets,Revenue per Share,NI per Share,outstandingShares,Revenue,Gross Profit,R&D Expenses,Op Expenses,Op Income,Net Income,Cash,Receivables,Inventory,Cur Assets,Tax Assets,LT Assets,Int Assets,Total Assets,Payables,Deferred Revenue Current,Deferred Revenue LongTerm,Cur Liab,LT Debt,LT Liab,Total Liab,SH Equity,cash_flow Operations,cash_flow Investing,cash_flow Financing,CAPEX,Fcash_flow,Dividends Paid,Gross Profit Margin,Op Margin,Int Coverage,Net Profit Margin,Dividend Yield,Current Ratio,Operating Cycle,Days of AP Outstanding,Cash Conversion Cycle,ROA,ROE,ROCE,PE,PS,PB,Price To FCF,PEG,EPS,Ticker
0,0,2022-07-28,528.59,536.51,520.58,533.61,533.61,"1,480,833.00","1,480,833.00",5.02,0.95,530.24,"July 28, 22",0.01,AVGO,2022-06-25,"2,022.00",Q2,"235,150.80",1.87,0.71,19.86,6.35,424.00,"8,103.00","5,439.00","1,261.00","2,045.00","3,394.00","2,590.00","9,005.00","3,083.00","1,668.00","14,810.00",0.00,"56,909.00","9,241.00","71,719.00","1,069.00",0.00,0.00,"6,910.00","39,164.00","43,819.00","50,729.00","20,963.00","4,243.00",-310.00,"-5,147.00",-85.00,"4,158.00","-1,750.00",0.67,0.42,6.55,0.32,0.01,2.14,90.59,36.11,54.48,0.04,0.12,0.05,22.70,29.02,11.22,56.55,2.49,6.35,AVGO
1,1,2022-07-27,518.20,530.19,516.93,527.79,527.79,"2,570,045.00","2,570,045.00",9.59,1.85,524.97,"July 27, 22",0.02,AVGO,2022-03-26,"2,022.00",Q1,"241,382.56",1.71,0.69,18.70,6.00,429.00,"7,706.00","5,049.00","1,206.00","1,924.00","3,108.00","2,472.00","10,219.00","2,539.00","1,520.00","15,341.00",0.00,"57,883.00","10,244.00","73,224.00","1,078.00","3,027.00",0.00,"6,287.00","39,205.00","43,943.00","50,230.00","22,994.00","3,486.00",-309.00,"-5,121.00",-101.00,"3,385.00","-1,764.00",0.66,0.40,7.64,0.32,0.01,2.44,81.14,36.51,44.63,0.03,0.11,0.05,24.41,31.32,10.50,71.31,0.97,5.82,AVGO
2,2,2022-07-26,514.63,516.81,508.92,511.09,511.09,"1,794,706.00","1,794,706.00",-3.54,-0.69,512.27,"July 26, 22",-0.01,AVGO,2021-12-25,"2,021.00",Q4,"217,556.60",1.58,0.67,17.98,4.83,430.00,"7,407.00","4,638.00","1,200.00","2,031.00","2,581.00","1,989.00","12,163.00","2,071.00","1,297.00","16,586.00",0.00,"58,984.00","11,374.00","75,570.00","1,086.00","2,619.00",566.00,"6,281.00","39,440.00","44,300.00","50,581.00","24,989.00","3,541.00",50.00,"-2,533.00",-88.00,"3,453.00","-1,561.00",0.63,0.35,5.95,0.27,0.01,2.64,67.32,35.30,32.02,0.03,0.08,0.04,27.34,29.37,8.71,63.01,4.44,4.65,AVGO
3,3,2022-07-25,510.46,513.32,506.53,513.11,513.11,"1,502,740.00","1,502,740.00",2.65,0.52,510.99,"July 25, 22",0.01,AVGO,2021-09-25,"2,021.00",Q3,"199,203.48",1.65,0.68,16.49,4.56,429.00,"6,778.00","4,197.00","1,205.00","2,045.00","2,126.00","1,876.00","11,105.00","2,234.00","1,160.00","15,636.00",0.00,"60,244.00","12,719.00","75,880.00",968.00,"3,008.00",620.00,"6,501.00","40,178.00","45,012.00","51,513.00","24,367.00","3,541.00",-47.00,"-1,907.00",-115.00,"3,426.00","-1,556.00",0.62,0.31,5.12,0.28,0.01,2.41,70.11,33.75,36.36,0.02,0.08,0.03,26.55,29.39,8.18,58.14,1.00,4.38,AVGO
4,4,2022-07-22,518.81,519.79,506.68,512.52,512.52,"1,784,265.00","1,784,265.00",-6.29,-1.21,513.00,"July 22, 22",-0.01,AVGO,2021-06-26,"2,021.00",Q2,"184,107.27",1.68,0.68,16.16,3.65,429.00,"6,610.00","4,057.00","1,238.00","2,057.00","1,975.00","1,493.00","9,518.00","2,425.00","1,004.00","14,245.00",0.00,"61,279.00","14,068.00","75,524.00",830.00,"3,409.00",688.00,"6,436.00","40,160.00","45,122.00","51,558.00","23,966.00","3,569.00",-126.00,"-3,477.00",-126.00,"3,443.00","-1,552.00",0.61,0.30,4.24,0.23,0.01,2.21,68.41,29.26,39.15,0.02,0.06,0.03,30.83,27.85,7.68,53.47,3.79,3.46,AVGO


In [188]:
def total_yield(ticker):
    """ 75%* Indicated Annual Dividend + 25%* YoY %chg in quarterly share count"""
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df['DivYieldAnnual'] = df['Dividend Yield']*4
    df['share_count_change'] = df.groupby(['Quarter'])['outstandingShares'].pct_change().shift(-4)
    df['TotalYield'] = .75*df['DivYieldAnnual'] + (.25*df['share_count_change'])
    return df

In [191]:
total_yield('MSFT').head()

,Unnamed: 0,Calendar_Year,Quarter,Mkt Cap,Debt to Equity,Debt to Assets,Revenue per Share,NI per Share,outstandingShares,Revenue,Gross Profit,R&D Expenses,Op Expenses,Op Income,Net Income,Cash,Receivables,Inventory,Cur Assets,Tax Assets,LT Assets,Int Assets,Total Assets,Payables,Deferred Revenue Current,Deferred Revenue LongTerm,Cur Liab,LT Debt,LT Liab,Total Liab,SH Equity,cash_flow Operations,cash_flow Investing,cash_flow Financing,CAPEX,Fcash_flow,Dividends Paid,Gross Profit Margin,Op Margin,Int Coverage,Net Profit Margin,Dividend Yield,Current Ratio,Operating Cycle,Days of AP Outstanding,Cash Conversion Cycle,ROA,ROE,ROCE,PE,PS,PB,Price To FCF,PEG,EPS,Ticker,DivYieldAnnual,share_count_change,TotalYield
0,2022-06-25,2022,Q4,"1,919,547.42",0.35,0.54,6.94,2.24,"7,506.00","51,865.00","35,436.00","6,849.00","14,902.00","20,534.00","16,740.00","13,931.00","44,261.00","3,742.00","169,684.00",0.00,"195,156.00","11,298.00","364,840.00","19,000.00","45,538.00","2,870.00","95,082.00","58,521.00","103,216.00","198,298.00","166,542.00","24,629.00","-8,591.00","-13,269.00","-6,871.00","17,758.00","-4,632.00",0.68,0.40,NaN,0.32,0.00,1.78,97.30,104.08,-6.78,0.05,0.10,0.08,28.67,37.01,11.53,108.09,63.93,2.24,MSFT,0.01,0.01,0.01
1,2022-03-26,2022,Q3,"2,310,166.81",0.37,0.53,6.59,2.23,"7,534.00","49,360.00","33,745.00","6,306.00","13,381.00","20,364.00","16,728.00","12,498.00","32,613.00","3,296.00","153,922.00",0.00,"190,685.00","11,348.00","344,607.00","16,085.00","34,027.00","2,769.00","77,439.00","59,534.00","104,244.00","181,683.00","162,924.00","25,386.00","-16,171.00","-17,345.00","-5,340.00","20,046.00","-4,645.00",0.68,0.41,40.49,0.34,0.00,1.99,78.46,92.71,-14.25,0.05,0.10,0.08,34.53,46.80,14.18,115.24,-3.20,2.23,MSFT,0.01,0.01,0.01
2,2021-12-25,2022,Q2,"2,524,081.65",0.37,0.53,6.89,2.50,"7,555.00","51,728.00","34,768.00","5,758.00","12,521.00","22,247.00","18,765.00","20,604.00","33,520.00","3,019.00","174,188.00",199.00,"166,201.00","7,462.00","340,389.00","15,314.00","34,001.00","2,768.00","77,510.00","59,034.00","102,869.00","180,379.00","160,010.00","14,480.00","-1,161.00","-11,986.00","-5,865.00","8,615.00","-4,652.00",0.67,0.43,42.38,0.36,0.00,2.25,74.34,81.27,-6.92,0.06,0.12,0.08,33.63,48.80,15.77,292.99,-3.99,2.50,MSFT,0.01,0.01,0.01
3,2021-09-25,2022,Q1,"2,118,065.06",0.40,0.55,6.03,2.73,"7,567.00","45,317.00","31,671.00","5,599.00","11,433.00","20,238.00","20,505.00","16,845.00","27,349.00","3,411.00","174,326.00",0.00,"161,092.00","7,794.00","335,418.00","14,832.00","38,465.00","2,550.00","80,528.00","60,089.00","102,912.00","183,440.00","151,978.00","24,540.00","-3,250.00","-16,276.00","-5,810.00","18,730.00","-4,206.00",0.70,0.45,37.55,0.45,0.00,2.16,76.81,97.82,-21.01,0.06,0.13,0.08,25.82,46.74,13.94,113.08,1.05,2.73,MSFT,0.01,0.01,0.01
4,2021-06-26,2021,Q4,"2,039,064.25",0.42,0.57,6.13,2.19,"7,581.00","46,152.00","32,161.00","5,687.00","13,066.00","19,095.00","16,458.00","11,748.00","38,043.00","2,636.00","184,406.00",0.00,"149,373.00","7,800.00","333,779.00","15,163.00","41,525.00","2,616.00","88,657.00","59,703.00","103,134.00","191,791.00","141,988.00","22,710.00","-10,853.00","-11,371.00","-6,452.00","16,258.00","-4,214.00",0.70,0.41,34.53,0.36,0.00,2.08,91.14,97.54,-6.40,0.05,0.12,0.08,30.97,44.18,14.36,125.42,4.54,2.19,MSFT,0.01,0.01,0.01
